# Lecture 10: Classification Part 1

### Classification using scikit-learn (with pandas)

Classification Algorithms covered:
1. k-Nearest Neighbors
2. Decision Trees / Random Forest
3. Logistic Regression

Notebook created by Jennifer Widom, modified by Lisa Wang.

In [2]:
import csv
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

-pandas 기능
train = cities[(cities.country=='Norway') | (cities.country=='France') | (cities.country=='Turkey')] 
CT_dataframe['temp'].max() temps만 찾아서 뒤에 함수도 가능. 
합치기도 가능
sql기능 다 가능

-list에서 key(label) 찾아 검색하는 거, 
앞에 7장에서 sorted_transactions = collections.OrderedDict(sorted(transactions.items()))
해서 row['key']로 검색 가능

In [3]:
# Read Cities.csv into dataframe, add column for temperature category
# Note: For a dataframe D and integer i, D.ix[i] is the i-th row of D ix없어진대서 iloc으로 바꿈
f = open('Cities.csv','rU')
cities = pd.read_csv(f)
cats = []
for i in range(len(cities)):
    if cities.iloc[i]['temperature'] < 5: # iloc은 원래 리스트 안에 항목 쓰듯이  loc은 그 안에 label 찾아서. 그게 똑같은 거 까지
        cats.append('cold')
    elif cities.iloc[i]['temperature'] < 9:
        cats.append('cool')
    elif cities.iloc[i]['temperature'] < 15:
        cats.append('warm')
    else: cats.append('hot')
cities['category'] = cats #cat를 citiies에 새로운 항목으로 넣음
print "cold:", len(cities[(cities.category == 'cold')])#판다 데이터 프레임의 기능
print "cool:", len(cities[(cities.category == 'cool')])
print "warm:", len(cities[(cities.category == 'warm')])
print "hot:", len(cities[(cities.category == 'hot')])


cold: 17
cool: 92
warm: 79
hot: 25


In [4]:
print cities[:2]

       city         country  latitude  longitude  temperature category
0   Aalborg         Denmark     57.03       9.92         7.52     cool
1  Aberdeen  United Kingdom     57.17      -2.08         8.10     cool


In [5]:
# Create training and test sets for cities data
num_items = len(cities)
percent_train = 0.85 #85%를 traning set으로 쓰겠다
num_train = int(num_items*percent_train)
num_test = num_items - num_train
print 'Training set', num_train, 'items'
print'Test set', num_test, 'items'
citiesTrain = cities[0:num_train]
citiesTest = cities[num_train:]

Training set 181 items
Test set 32 items


http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
    train_test_split(dataset, 얼마나 test 할건지,shuffle true or false, random state )
    import numpy as np
>>> from sklearn.model_selection import train_test_split
>>> X, y = np.arange(10).reshape((5, 2)), range(5)
X_train, X_test, y_train, y_test = train_test_split(
...     X, y, test_size=0.33, random_state=42)

In [6]:
print citiesTrain[:10]

        city         country  latitude  longitude  temperature category
0    Aalborg         Denmark     57.03       9.92         7.52     cool
1   Aberdeen  United Kingdom     57.17      -2.08         8.10     cool
2     Abisko          Sweden     63.35      18.83         0.20     cold
3      Adana          Turkey     36.99      35.32        18.67      hot
4   Albacete           Spain     39.00      -1.87        12.62     warm
5  Algeciras           Spain     36.13      -5.47        17.38      hot
6     Amiens          France     49.90       2.30        10.17     warm
7  Amsterdam     Netherlands     52.35       4.92         8.93     cool
8     Ancona           Italy     43.60      13.50        13.52     warm
9    Andorra         Andorra     42.50       1.52         9.60     warm


Pandas: Note that you can access individual rows by their row index. E.g.

In [7]:
citiesTrain.ix[0] #첫 요소를 찍어봄

C:\Users\ChaeYoon\Anaconda3\envs\py27\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


city           Aalborg
country        Denmark
latitude         57.03
longitude         9.92
temperature       7.52
category          cool
Name: 0, dtype: object

In [8]:
print citiesTest[:10]

             city         country  latitude  longitude  temperature category
181         Sivas          Turkey     39.75      37.03         8.05     cool
182        Skopje       Macedonia     42.00      21.43         9.36     warm
183         Split         Croatia     43.52      16.47        12.46     warm
184  Stara Zagora        Bulgaria     42.42      25.62        10.90     warm
185     Stavanger          Norway     58.97       5.68         5.53     cool
186     Stockholm          Sweden     59.35      18.10         6.26     cool
187          Sumy         Ukraine     50.92      34.78         6.28     cool
188       Swansea  United Kingdom     51.63      -3.95         9.73     warm
189        Szeged         Hungary     46.25      20.15        10.34     warm
190       Tallinn         Estonia     59.43      24.73         4.82     cold


### K-nearest-neighbors classification

In [9]:
# Predict temperature category from other features
features = ['longitude', 'latitude']

# Create classfier
neighbors = 7 # Number of neighbors to consider for k nearest neighbor classification k를 7로
classifier = KNeighborsClassifier(n_neighbors=neighbors)

# Train the classifier on training data
classifier.fit(citiesTrain[features], citiesTrain['category']) #input, lable
#fit함수는 fit(x,y) x를 트레이닝데이터로 y를 타겟으로 씀
# Make predictions on training data
train_predictions = classifier.predict(citiesTrain[features])
#predict(data) 데이터에 대한 예상된 labeled 디이터 줌
# Make predictions on test data
test_predictions = classifier.predict(citiesTest[features])

num_train = len(citiesTrain)
num_test = len(citiesTest)
# print classifier.kneighbors(citiesTest[features]) 그 주변의 포인트들 반환
# Calculate training accuracy
train_correct = 0
for i in range(num_train):
    print 'Predicted:', train_predictions[i], ' Actual:', citiesTrain.ix[i]['category']
    if train_predictions[i] == citiesTrain.ix[i]['category']: train_correct +=1
print 'Training Accuracy:', float(train_correct)/float(num_train)
print ""

# Calculate test accuracy
test_correct = 0
for i in range(num_test):
    print 'Predicted:', test_predictions[i], ' Actual:', citiesTest.ix[num_train + i]['category']
    if test_predictions[i] == citiesTest.ix[num_train + i]['category']: test_correct +=1
print 'Test Accuracy:', float(test_correct)/float(num_test)
# Comment out print, try other values for neighbors, other features

Predicted: cool  Actual: cool
Predicted: cool  Actual: cool
Predicted: cold  Actual: cold
Predicted: warm  Actual: hot
Predicted: hot  Actual: warm
Predicted: hot  Actual: hot
Predicted: warm  Actual: warm
Predicted: cool  Actual: cool
Predicted: warm  Actual: warm
Predicted: warm  Actual: warm
Predicted: warm  Actual: warm
Predicted: warm  Actual: warm
Predicted: warm  Actual: warm
Predicted: warm  Actual: warm
Predicted: hot  Actual: hot
Predicted: cool  Actual: cold
Predicted: cool  Actual: cool
Predicted: hot  Actual: hot
Predicted: cool  Actual: cool
Predicted: cool  Actual: cool
Predicted: warm  Actual: hot
Predicted: hot  Actual: hot
Predicted: cool  Actual: cool
Predicted: warm  Actual: warm
Predicted: cool  Actual: cool
Predicted: warm  Actual: warm
Predicted: cool  Actual: warm
Predicted: cool  Actual: cold
Predicted: cool  Actual: cool
Predicted: cool  Actual: cool
Predicted: cool  Actual: cool
Predicted: cool  Actual: cool
Predicted: warm  Actual: warm
Predicted: warm  Actu

C:\Users\ChaeYoon\Anaconda3\envs\py27\lib\site-packages\ipykernel_launcher.py:31: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


### <font color="green">Your Turn: K-nearest-neighbors on World Cup Data</font>

In [12]:
# Predict position from one or more of minutes, shots, passes, tackles, saves.
# This cell does all the set-up, including reordering the data to avoid team bias.
f = open('Players.csv','rU')

players = pd.read_csv(f)
# ascending=True by default from sklearn.utils import shuffle
 #섞는 방법
            #from sklearn.utils import shuffle
            #players = shuffle(players)

players = players.sort_values(by='surname')#원래 팀별로 나눠져 있는데 그러면 편중되게 들어가니까 이름별로 재정렬
#print players.iloc[0:10]
# Drop indices
players = players.reset_index(drop=True)
#print players.iloc[0:10]
#print players[0:10]
num_items = len(players)
percent_train = 0.95
num_train = int(num_items*percent_train)
num_test = num_items - num_train
print 'Training set', num_train, 'items'
print'Test set', num_test, 'items'
playersTrain = players[0:num_train]
playersTest = players[num_train:]

                 surname          team    position  minutes  shots  passes  \
90                 Matip      Cameroon    defender       63      0      37   
41                Carney     Australia    defender      180      0      56   
165                Henry        France     forward       53      1      21   
448              Holosko      Slovakia     forward       29      0      11   
433              Lukovic        Serbia    defender      164      1      98   
560            Fernandez       Uruguay  midfielder       75      1      17   
484              Khumalo  South Africa    defender      270      2      74   
385         Ri Myong-Guk   North Korea  goalkeeper      270      0      46   
427            Jovanovic        Serbia  midfielder      245      4      65   
302              Aguilar        Mexico    defender       55      0      31   
33             Rodriguez     Argentina    defender       90      1      72   
308   Giovani dos Santos        Mexico  midfielder      360     

In [22]:
features = ['minutes', 'shots', 'passes', 'tackles', 'saves']
# Predict a player's position ( playersTrain['position'] ) 

# Create classfier
neighbors = 11
classifier = KNeighborsClassifier(n_neighbors=neighbors)

# Train the classifier on training data
classifier.fit(playersTrain[features], playersTrain['position'])

# Make predictions on training data
train_predictions = classifier.predict(playersTrain[features])

# Make predictions on test data
test_predictions = classifier.predict(playersTest[features])

num_train = len(playersTrain)
num_test = len(playersTest)
# Calculate training accuracy
train_correct = 0
for i in range(num_train):
     print 'Predicted:', train_predictions[i], ' Actual:', playersTrain.ix[i]['position']
    if train_predictions[i] == playersTrain.ix[i]['position']: train_correct +=1
print 'Training Accuracy:', float(train_correct)/float(num_train)
print ""

# Calculate test accuracy
test_correct = 0
for i in range(num_test):
    print 'Predicted:', test_predictions[i], ' Actual:', playersTest.ix[num_train + i]['position']
    if test_predictions[i] == playersTest.ix[num_train + i]['position']: test_correct +=1
print 'Test Accuracy:', float(test_correct)/float(num_test)


Training Accuracy: 0.623008849558

Predicted: forward  Actual: defender
Predicted: midfielder  Actual: forward
Predicted: forward  Actual: forward
Predicted: defender  Actual: midfielder
Predicted: forward  Actual: forward
Predicted: defender  Actual: midfielder
Predicted: defender  Actual: midfielder
Predicted: forward  Actual: forward
Predicted: defender  Actual: midfielder
Predicted: midfielder  Actual: midfielder
Predicted: defender  Actual: defender
Predicted: defender  Actual: midfielder
Predicted: forward  Actual: forward
Predicted: forward  Actual: forward
Predicted: midfielder  Actual: midfielder
Predicted: midfielder  Actual: midfielder
Predicted: midfielder  Actual: midfielder
Predicted: forward  Actual: defender
Predicted: defender  Actual: defender
Predicted: midfielder  Actual: defender
Predicted: midfielder  Actual: midfielder
Predicted: defender  Actual: defender
Predicted: forward  Actual: forward
Predicted: defender  Actual: midfielder
Predicted: defender  Actual: mid

## Decision tree classification

In [24]:
# Predict temperature category from other features
features = ['longitude','latitude']

# Create classifier
split = 10 #split횟수를 10번으로 함
# min_samples_split : number of samples required to split an internal node
# if < 10, then stop splitting 10갸 미만으로 들어가 있으면 split안함
dt = DecisionTreeClassifier(min_samples_split=split) # parameter is optional

# Train the classifier on training data 훈련
dt.fit(citiesTrain[features], citiesTrain['category'])#입력, 출력

# Make predictions on training data 트레인데이터 예측
#리스트로 에측 결과 들어가 있음
train_predictions = dt.predict(citiesTrain[features])

# Make predictions on test data 테스트 데이터 예측
test_predictions = dt.predict(citiesTest[features])

num_train = len(citiesTrain)
num_test = len(citiesTest)
# Calculate training accuracy
train_correct = 0
for i in range(num_train):
#     print 'Predicted:', train_predictions[i], ' Actual:', citiesTrain.ix[i]['category']
    if train_predictions[i] == citiesTrain.ix[i]['category']: train_correct +=1
print 'Training Accuracy:', float(train_correct)/float(num_train)
print ""

# Calculate test accuracy
test_correct = 0
for i in range(num_test):
#     print 'Predicted:', test_predictions[i], ' Actual:', citiesTest.ix[num_train + i]['category']
    if test_predictions[i] == citiesTest.ix[num_train + i]['category']: test_correct +=1
print 'Test Accuracy:', float(test_correct)/float(num_test)

Training Accuracy: 0.933701657459

Test Accuracy: 0.75


### "Forest" of decision trees

In [43]:
# Predict temperature category from other features
features = ['longitude', 'latitude']

# Create classifier
trees = 10 # Try other values for trees
rf = RandomForestClassifier(n_estimators=trees)

# Train the classifier on training data
rf.fit(citiesTrain[features], citiesTrain['category'])#입력 출력

# Make predictions on training data
train_predictions = rf.predict(citiesTrain[features])

# Make predictions on test data
test_predictions = rf.predict(citiesTest[features])

num_train = len(citiesTrain)
num_test = len(citiesTest)
# Calculate training accuracy
train_correct = 0
for i in range(num_train):
#     print 'Predicted:', train_predictions[i], ' Actual:', citiesTrain.ix[i]['category']
    if train_predictions[i] == citiesTrain.ix[i]['category']: train_correct +=1
print 'Training Accuracy:', float(train_correct)/float(num_train)
print ""

# Calculate test accuracy
test_correct = 0
for i in range(num_test):
#     print 'Predicted:', test_predictions[i], ' Actual:', citiesTest.ix[num_train + i]['category']
    if test_predictions[i] == citiesTest.ix[num_train + i]['category']: test_correct +=1
print 'Test Accuracy:', float(test_correct)/float(num_test)

Training Accuracy: 0.983425414365

Test Accuracy: 0.78125


### <font color="green">Your Turn: Decision tree and forest of trees on World Cup Data</font>

In [53]:
# SINGLE TREE
# Predict position from one or more of minutes, shots, passes, tackles, saves.
# Try different features and different values for min_samples_split.
# What's the highest accuracy you can get?
features = ['minutes', 'shots', 'passes', 'tackles', 'saves']

# Create classifier
split = 10
dt = DecisionTreeClassifier(min_samples_split=split) # parameter is optional

# Train the classifier on training data
dt.fit(playersTrain[features], playersTrain['position'])

# Make predictions on training data
train_predictions = dt.predict(playersTrain[features])

# Make predictions on test data
test_predictions = dt.predict(playersTest[features])

num_train = len(playersTrain)
num_test = len(playersTest)
# Calculate training accuracy
train_correct = 0 #range(2,9)하면 2부터 9
for i in range(num_train):
#     print 'Predicted:', train_predictions[i], ' Actual:', playersTrain.ix[i]['position']
    if train_predictions[i] == playersTrain.ix[i]['position']: train_correct +=1
print 'Training Accuracy:', float(train_correct)/float(num_train)
print ""

# Calculate test accuracy
test_correct = 0
for i in range(num_test):
#     print 'Predicted:', test_predictions[i], ' Actual:', playersTest.ix[num_train + i]['position']
    if test_predictions[i] == playersTest.ix[num_train + i]['position']: test_correct +=1
print 'Test Accuracy:', float(test_correct)/float(num_test)

Training Accuracy: 0.844247787611

Test Accuracy: 0.466666666667


In [56]:
# FOREST OF TREES
# Predict position from one or more of minutes, shots, passes, tackles, saves.
# Try different values for n_estimators.
# What's the highest accuracy you can get?
features = ['minutes', 'shots', 'passes', 'tackles', 'saves']

# Create classifier
trees = 10
rf = RandomForestClassifier(n_estimators=trees)

# Train the classifier on training data
rf.fit(playersTrain[features], playersTrain['position'])

# Make predictions on training data
train_predictions = rf.predict(playersTrain[features])

# Make predictions on test data
test_predictions = rf.predict(playersTest[features])

num_train = len(playersTrain)
num_test = len(playersTest)
# Calculate training accuracy
train_correct = 0
for i in range(num_train):
#     print 'Predicted:', train_predictions[i], ' Actual:', playersTrain.ix[i]['position']
    if train_predictions[i] == playersTrain.ix[i]['position']: train_correct +=1
print 'Training Accuracy:', float(train_correct)/float(num_train)
print ""

# Calculate test accuracy
test_correct = 0
for i in range(num_test):
#     print 'Predicted:', test_predictions[i], ' Actual:', playersTest.ix[num_train + i]['position']
    if test_predictions[i] == playersTest.ix[num_train + i]['position']: test_correct +=1
print 'Test Accuracy:', float(test_correct)/float(num_test)

Training Accuracy: 0.969911504425

Test Accuracy: 0.6


### Logistic Regression Classification

In [61]:
# Predict temperature category from other features
features = ['longitude', 'latitude']

# Create classifier
lg = LogisticRegression()

# Train the classifier on training data
lg.fit(citiesTrain[features], citiesTrain['category'])

# Make predictions on training data
train_predictions = lg.predict(citiesTrain[features])

# Make predictions on test data
test_predictions = lg.predict(citiesTest[features])

num_train = len(citiesTrain)
num_test = len(citiesTest)
# Calculate training accuracy
train_correct = 0
for i in range(num_train):
#     print 'Predicted:', train_predictions[i], ' Actual:', citiesTrain.ix[i]['category']
    if train_predictions[i] == citiesTrain.ix[i]['category']: train_correct +=1
print 'Training Accuracy:', float(train_correct)/float(num_train)
print ""

# Calculate test accuracy
test_correct = 0
for i in range(num_test):
#     print 'Predicted:', test_predictions[i], ' Actual:', citiesTest.ix[num_train + i]['category']
    if test_predictions[i] == citiesTest.ix[num_train + i]['category']: test_correct +=1
print 'Test Accuracy:', float(test_correct)/float(num_test)

Training Accuracy: 0.541436464088

Test Accuracy: 0.4375


### <font color="green">Your Turn: Logistic Regression on World Cup Data</font>

In [64]:
# FOREST OF TREES
# Predict position from one or more of minutes, shots, passes, tackles, saves.
# Try different values for n_estimators.
# What's the highest accuracy you can get?
features = ['minutes', 'shots', 'passes', 'tackles', 'saves']

# Create classifier
lg = LogisticRegression()

# Train the classifier on training data
lg.fit(playersTrain[features], playersTrain['position'])

# Make predictions on training data
train_predictions = lg.predict(playersTrain[features])

# Make predictions on test data
test_predictions = lg.predict(playersTest[features])

num_train = len(playersTrain)
num_test = len(playersTest)
# Calculate training accuracy
train_correct = 0
for i in range(num_train):
#     print 'Predicted:', train_predictions[i], ' Actual:', playersTrain.ix[i]['position']
    if train_predictions[i] == playersTrain.ix[i]['position']: train_correct +=1
print 'Training Accuracy:', float(train_correct)/float(num_train)
print ""

# Calculate test accuracy
test_correct = 0
for i in range(num_test):
#     print 'Predicted:', test_predictions[i], ' Actual:', playersTest.ix[num_train + i]['position']
    if test_predictions[i] == playersTest.ix[num_train + i]['position']: test_correct +=1
print 'Test Accuracy:', float(test_correct)/float(num_test)

Training Accuracy: 0.647787610619

Test Accuracy: 0.733333333333
